<img src="https://toppng.com/uploads/preview/linkedin-logo-png-photo-116602552293wtc4qogql.png" width="20" height="20" /> [Bharath Hemachandran](https://www.linkedin.com/in/bharath-hemachandran/)

# 🔌 Phase 2: Groq + One MCP (Hugging Face)

Add **one MCP server** so the model can use **tools** (e.g. search Hugging Face). No approval flow—<code>require_approval: "never"</code>—so the run completes in one go.

<div style="background: #fce4ec; padding: 14px; border-radius: 8px; border-left: 4px solid #c2185b;">
<strong>🎯 What you'll do:</strong> Call the Groq API with a single MCP tool (Hugging Face). The model will use the tool and return text + tool outputs.
</div>

### 📋 Notebook objective (table of contents)

This notebook covers:
- **Setup** — Install OpenAI client
- **API keys** — GROQ_API_KEY (required), HF_TOKEN (optional)
- **Define the MCP tool** — type, server URL, require_approval: "never"
- **Call Groq with the MCP tool** — One request; model uses tool and returns text
- **Output items** — Types in <code>response.output</code> (e.g. mcp_call)
- **Additional reading** — MCP spec and Groq docs


## 🔧 Setup (run once)

Install **openai**. On Colab, run this cell first.

In [ ]:
!pip install -q openai

### 🔑 API keys

- **GROQ_API_KEY** — required. Get it at [console.groq.com](https://console.groq.com/keys).<br>
- **HF_TOKEN** (optional) — for Hugging Face MCP; improves rate limits. Get it at [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens).

In [ ]:
import os
from getpass import getpass

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass("Paste your GROQ_API_KEY: ")
if not os.environ.get("HF_TOKEN"):
    tok = getpass("Paste your HF_TOKEN (or press Enter to skip): ")
    if tok:
        os.environ["HF_TOKEN"] = tok

from openai import OpenAI

def get_groq_client():
    return OpenAI(
        api_key=os.environ["GROQ_API_KEY"],
        base_url="https://api.groq.com/openai/v1",
    )

HF_TOKEN = os.environ.get("HF_TOKEN")
print("✅ Groq client ready." if os.environ.get("GROQ_API_KEY") else "Need GROQ_API_KEY.")

## 🔧 Define the MCP tool

One tool entry: <code>type: "mcp"</code>, server URL, and <code>require_approval: "never"</code> so the run doesn't wait for approval.

In [ ]:
mcp_tool = {
    "type": "mcp",
    "server_label": "Huggingface",
    "server_url": "https://huggingface.co/mcp",
    "server_description": "Search and access AI models from Hugging Face",
    "require_approval": "never",
}
if HF_TOKEN:
    mcp_tool["headers"] = {"Authorization": f"Bearer {HF_TOKEN}"}

print("MCP tool: Hugging Face (require_approval: never)")

## 🚀 Call Groq with the MCP tool

We use a model that supports tools (e.g. <code>openai/gpt-oss-120b</code>). The model may issue **tool calls**; with <code>require_approval: "never"</code> they run automatically.

In [ ]:
client = get_groq_client()

response = client.responses.create(
    model="openai/gpt-oss-120b",
    input="What models are trending on Hugging Face? List up to 3.",
    tools=[mcp_tool],
)

print("--- output_text ---")
print(response.output_text)

## 📦 Output items (types)

The <code>response.output</code> list can contain text and tool-related items (e.g. <code>mcp_call</code>).

In [ ]:
print("--- output item types ---")
for item in response.output:
    t = getattr(item, "type", item)
    print(t)

In [ ]:
print("✅ Phase 2 complete. Next: Phase 3 (multi-step MCP agent).")

## 📚 Additional reading

**YouTube (verified)**  
- [The Model Context Protocol (MCP)](https://www.youtube.com/watch?v=CQywdSdi5iA) — Anthropic: MCP intro and why it matters.  
- [Model Context Protocol, clearly explained](https://www.youtube.com/watch?v=7j_NE6Pjv-E) — Why MCP matters for connecting AI to tools.

**Blogs (popular)**  
- [Model Context Protocol – Specification](https://modelcontextprotocol.io/specification/latest) — Official MCP spec: tools, resources, lifecycle.  
- [Groq API Reference](https://console.groq.com/docs) — Groq docs: Responses API, tools, MCP connectors.